In [1]:
import scipy as sp
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

plt.style.use('seaborn-dark-palette')

### Bayesian A/B

In [ ]:
def bayes_AB(data , prior):
    A = data['A']
    B = data['B']
    posterior = 
    return posterior

### Frequentist A/B

In [ ]:
def ttest(data,ispaired = True, istwosided = False): 
    return pg.ttest(data['A'], data['B'], paired=ispaired, tail='two-sided')